In [ ]:
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist

nltk.download('stopwords')
nltk.download('punkt')

# Load spaCy model for NER
nlp = spacy.load("en_core_web_sm")

def generate_summary(input_text, style_text, context_window=3):
    input_sentences = sent_tokenize(input_text)
    style_sentences = sent_tokenize(style_text)

    # Measure the length of the documents
    input_length = len(input_sentences)
    style_length = len(style_sentences)

    # Compute target lengths in a proportional way
    target_input_length = int(input_length / style_length * context_window)

    # Initialize summary list
    summaries = []

    for i in range(0, style_length, context_window):
        # Slice the second document within the context window
        slice_start = i
        slice_end = min(i + context_window, style_length)
        sliced_style_text = ' '.join(style_sentences[slice_start:slice_end])

        # Summarize the slice with no request for the size of the target
        summary = generate_simple_summary(sliced_style_text)
        summaries.append(summary)

    # Collate the summaries
    collated_summary = ' '.join(summaries)

    # Repeat shrinking activities until the summary size is within the context window
    while len(collated_summary) > target_input_length:
        collated_summary = generate_simple_summary(collated_summary)

    # Generate the query based on the final summary
    query = generate_query(collated_summary)
    
    return collated_summary, query

def generate_query(summary):
    # Extract named entities using spaCy
    doc = nlp(summary)
    named_entities = [ent.text for ent in doc.ents]

    # Analyze sentiment of the summary
    sentiment_score = analyze_sentiment(summary)

    # Construct a query based on named entities and sentiment
    query = f"Entities: {', '.join(named_entities)} | Sentiment: {sentiment_score}"

    return query

def analyze_sentiment(text):
    # Placeholder sentiment analysis (you can replace this with a more advanced sentiment analysis tool)
    # This is just an example, replace it with a proper sentiment analysis library or API
    return "Positive" if "good" in text.lower() else "Negative"

def generate_simple_summary(text):
    # Tokenize and preprocess text
    sentences = sent_tokenize(text)
    words = [word.lower() for sentence in sentences for word in word_tokenize(sentence) if word.isalnum()]
    stems = [PorterStemmer().stem(word) for word in words if word not in stopwords.words('english')]

    # Calculate frequency distribution
    freq_dist = FreqDist(stems)

    # Extract key words based on frequency
    key_words = [word for word, freq in freq_dist.items() if freq > 1]

    # Create a summary sentence by selecting key words
    summary = ' '.join([word for sentence in sentences for word in word_tokenize(sentence) if word.lower() in key_words])

    return summary

# User input for input text and style text
input_text = input("Enter the input text: ")
style_text = input("Enter the style text: ")

# Specify the context window size
context_window = 3

# Generate summary for the input text using the style of the provided text
result_summary, query = generate_summary(input_text, style_text, context_window)
print("Final Summary:", result_summary)
print("Generated Query:", query)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leulfeven\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leulfeven\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
